In [ ]:
import numpy as np
import matplotlib  
matplotlib.use('Qt5Agg')
from matplotlib import pyplot as plt
import cv2
import tensorflow as tf

%matplotlib qt5

In [ ]:
class PlotLearning(tf.keras.callbacks.Callback):
    """
    Callback to plot the learning curves of the model during training.
    """
    def on_train_begin(self, logs={}):
        self.metrics = {}
        if logs is not None:
            for metric in logs:
                self.metrics[metric] = []
        
        f, self.axs = plt.subplots(1, 1, figsize=(15,5))

    def on_epoch_end(self, epoch, logs={}):
        # Storing metrics
        for metric in logs:
            if metric in self.metrics:
                self.metrics[metric].append(logs.get(metric))
            else:
                self.metrics[metric] = [logs.get(metric)]
        
        # Plotting
        metrics = [x for x in logs if 'val' not in x]
        
        #clear_output(wait=True)

        for i, metric in enumerate(metrics):
            
            if len(self.metrics[metric])%50 == 0:
            
                self.axs.cla()

                
                met = np.array(self.metrics[metric][-1000:])
                
                self.axs.plot(np.log(1+met), label='%s - %d'%(metric, epoch) )

                self.axs.legend()
                self.axs.grid()

                
        plt.tight_layout()
        plt.pause(0.0001)

In [ ]:
IMG_SIZE = (64, 64)

In [ ]:
H = plt.figure(figsize=(5,10))

capIn = cv2.VideoCapture('spring-mass-in.mov')
capIn.set(cv2.CAP_PROP_POS_FRAMES, 0)

capOut = cv2.VideoCapture('spring-mass-out.mov')
capOut.set(cv2.CAP_PROP_POS_FRAMES, 0)

if (capIn.isOpened()== False) or (capOut.isOpened()== False):
    print("Error opening video stream or file")

N = 350
    
IMGsIn = []    
IMGsOut = []    
    
i = 0
while capOut.isOpened() and capIn.isOpened() and i < N:
    # Capture frame-by-frame
    retIn, frameIn = capIn.read()
    retOut, frameOut = capOut.read()
    
    imgIn = cv2.resize(frameIn, dsize=IMG_SIZE)[:,:,[2,1,0]]
    imgOut = cv2.resize(frameOut, dsize=IMG_SIZE)[:,:,[2,1,0]]
    
    IMGsIn.append(imgIn/255.0)
    IMGsOut.append(imgOut/255.0)
    
    if retIn == True and retOut == True:
        plt.subplot(2,1,1)
        plt.cla()
        plt.imshow(imgIn)

        plt.subplot(2,1,2)
        plt.cla()
        plt.imshow(imgOut)
        
        plt.pause(0.001)
    else:
        break
        
    i += 1

    
capIn.release()
capOut.release()

#cv2.destroyAllWindows()    

IMGsIn = np.array(IMGsIn)
IMGsOut = np.array(IMGsOut)



In [ ]:
# NBatches = 5
# NPerBatch = N//NBatches

# IMGsInBatches = np.zeros( (NBatches, NPerBatch, *IMG_SIZE, 3) )
# IMGsOutBatches = np.zeros( (NBatches, NPerBatch, *IMG_SIZE, 3) )

# for i in range(NBatches):
#     IMGsInBatches[i,:,:,:,:] = IMGsIn[NPerBatch*i:NPerBatch*(i+1),:,:,:]
#     IMGsOutBatches[i,:,:,:,:] = IMGsOut[NPerBatch*i:NPerBatch*(i+1),:,:,:]  


IMGsIn = IMGsIn.reshape( (1, *IMGsIn.shape) )
IMGsOut = IMGsOut.reshape( (1, *IMGsOut.shape) )


In [ ]:
latentDim = 1
stateDim = 2

In [ ]:
modelEncoder = tf.keras.Sequential()

modelEncoder.add( tf.keras.layers.Input( (None, IMG_SIZE[1],IMG_SIZE[0],3) ) )

modelEncoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(8, (5, 5), padding = 'same', activation='tanh') ) )

#modelEncoder.add( tf.keras.layers.Dropout(0.05) )

#modelEncoder.add( tf.keras.layers.Convolution2D(4, (3, 3), padding = 'same', activation='tanh') )

#modelEncoder.add( tf.keras.layers.Dropout(0.05) )

modelEncoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Flatten() ) )

modelEncoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Dense(1, activation='linear' ) ) )

modelEncoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.BatchNormalization( ) )


In [ ]:
modelDynamic = tf.keras.Sequential()
modelDynamic.add( tf.keras.layers.Input( shape=(None,latentDim) ) )
modelDynamic.add( tf.keras.layers.SimpleRNN(stateDim, return_sequences = True, use_bias = False, activation = 'linear' , stateful=False  )
)
modelDynamic.add( tf.keras.layers.Dense( 1, activation = 'linear', use_bias=False  ) )

In [ ]:
modelDecoder = tf.keras.Sequential()

modelDecoder.add( tf.keras.layers.Input( (None, latentDim) ) )
                 
# modelDecoder.add( tf.keras.layers.Dense(400, activation='tanh') )
# #modelDecoder.add( tf.keras.layers.LeakyReLU())
# #modelDecoder.add( tf.keras.layers.Dropout(0.05) )

# modelDecoder.add( tf.keras.layers.Dense(200, activation='tanh') )
# #modelDecoder.add( tf.keras.layers.LeakyReLU())
# #modelDecoder.add( tf.keras.layers.Dropout(0.05) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Dense(8*8, activation='sigmoid') ) )
#modelDecoder.add( tf.keras.layers.LeakyReLU())
#modelDecoder.add( tf.keras.layers.Dropout(0.05) )

modelDecoder.add( tf.keras.layers.TimeDistributed(tf.keras.layers.Reshape( (8, 8, 1) ) ) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(4, (3, 3), padding = 'same', activation='tanh') ) )
#modelDecoder.add( tf.keras.layers.LeakyReLU())
#modelDecoder.add( tf.keras.layers.Dropout(0.15) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.UpSampling2D( (2, 2) ) ) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(4, (3, 3), padding = 'same', activation='tanh') ) )
#modelDecoder.add( tf.keras.layers.LeakyReLU())
#modelDecoder.add( tf.keras.layers.Dropout(0.15) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.UpSampling2D( (2, 2) ) ) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(4, (3, 3), padding = 'same', activation='tanh') ) )
#modelDecoder.add( tf.keras.layers.LeakyReLU())
#modelDecoder.add( tf.keras.layers.Dropout(0.15) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.UpSampling2D( (2, 2) ) ) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(4, (3, 3), padding = 'same', activation='tanh') ) )
#modelDecoder.add( tf.keras.layers.Dropout(0.15) )

modelDecoder.add( tf.keras.layers.TimeDistributed( tf.keras.layers.Convolution2D(3, (3, 3), padding = 'same', activation='sigmoid') ) )
#modelDecoder.add( tf.keras.layers.Dropout(0.05) )

In [ ]:
# clone models in order to train them to leard system coordinates (OUT = IN with no dynamics in between)

modelEncoderIn = tf.keras.models.clone_model(modelEncoder)
modelEncoderOut = tf.keras.models.clone_model(modelEncoder)
modelDecoderIn = tf.keras.models.clone_model(modelDecoder)
modelDecoderOut = tf.keras.models.clone_model(modelDecoder)

In [ ]:
modelEncoderIn.summary()

In [ ]:
# stich the encodes to the decoders to allow train for coordinates
#
#  For input video:
#  modelEncoderIn -> Will be the real encoder
#  modelDecoderIn -> Will be the dummy decoder just to train latent coordinates
#
#  For output video:
#  modelEncoderOut -> Will be the dummy encoder just to train latent coordinates
#  modelDecoderOut -> Will be the real decoder


modelIn = tf.keras.models.Sequential( [modelEncoderIn, modelDecoderIn] )
modelIn.build( input_shape = modelEncoder.inputs[0].shape )

modelOut = tf.keras.models.Sequential( [modelEncoderOut, modelDecoderOut] )
modelOut.build( input_shape = modelEncoder.inputs[0].shape )

In [ ]:
# Compile models

#optimizer = tf.optimizers.Adam( learning_rate=0.005, beta_1=0.2, beta_2=0.999, epsilon=1e-07 )

#optimizer = tf.optimizers.SGD(  learning_rate=0.9, momentum=0.5 )

optimizer = tf.optimizers.SGD(  learning_rate=2.9 )
loss = tf.keras.losses.MSE


modelIn.compile(loss=loss, optimizer=optimizer)
modelOut.compile(loss=loss, optimizer=optimizer)

In [ ]:
# Train latent coordinates for input video

plotCallbacks = [PlotLearning()]

modelIn.fit(IMGsIn, IMGsIn, batch_size=len(IMGsIn), epochs=60000, verbose=0, callbacks=plotCallbacks)

In [ ]:
# Train latent coordinates for output video

plotCallbacks = [PlotLearning()]

modelOut.fit(IMGsOut, IMGsOut, batch_size=len(IMGsOut), epochs=60000, verbose=0, callbacks=plotCallbacks)

In [ ]:
# Test latent training results

idx = 40

plt.figure()
Iorig = IMGsIn[:,idx,:,:,:]
Ipred = modelIn.predict(IMGsIn[:,idx:idx+1,:,:,:])[0,0,:,:,:]
plt.subplot(1,2,1)
plt.imshow( Iorig[0,:,:,:] )
plt.subplot(1,2,2)
plt.imshow( Ipred )

plt.figure()
Iorig = IMGsOut[:,idx,:,:,:]
Ipred = modelOut.predict(IMGsOut[:,idx:idx+1,:,:,:])[0,0,:,:,:]
plt.subplot(1,2,1)
plt.imshow( Iorig[0,:,:,:] )
plt.subplot(1,2,2)
plt.imshow( Ipred )



In [102]:
#modelIn = tf.keras.models.load_model('modelIn')

modelEncoderIn = tf.keras.models.load_model('modelEncoderIn')
modelDecoderIn = tf.keras.models.load_model('modelDecoderIn')

#modelOut = tf.keras.models.load_model('modelOut')

modelEncoderOut = tf.keras.models.load_model('modelEncoderOut')
modelDecoderOut = tf.keras.models.load_model('modelDecoderOut')

modelDynamic.save('modelDynamic')

INFO:tensorflow:Assets written to: modelDynamic\assets


In [ ]:
modelIn.save('modelIn')
modelEncoderIn.save('modelEncoderIn')
modelDecoderIn.save('modelDecoderIn')
modelOut.save('modelOut')
modelEncoderOut.save('modelEncoderOut')
modelDecoderOut.save('modelDecoderOut')

modelDynamic.save('modelDynamic')

In [ ]:
# compute latent coordinates

codesIn = modelEncoderIn( IMGsIn )
#codesOutIn = modelDynamic(codesInIn)
decodesIn = codesIn

codesOut = modelEncoderOut( IMGsOut)
#codesOutOut = modelDynamic(codesInOut)
decodesOut = codesOut

In [103]:
# for i in range(NBatches):
#     t = np.linspace(NPerBatch*i,NPerBatch*(i+1), NPerBatch)
plt.plot(codesIn.numpy().reshape( (N) ))
plt.plot(decodesOut.numpy().reshape( (N)))

In [ ]:
def animateCoordinates(IMGs, codes, model):
    
    plt.figure(figsize=(10,3))
    
    IMGsPred = model.predict(IMGs)
    i = 0
    for img, imgPred in zip(IMGs[0,:,:,:,:], IMGsPred[0,:,:,:,:]):
        plt.subplot(1,4,1)
        plt.cla()
        plt.imshow(img)

        plt.subplot(1,4,2)
        plt.cla()
        plt.imshow(imgPred)
        
        plt.subplot(1,2,2)
        plt.cla()
        plt.plot(codes.numpy()[0,:,0])
        plt.plot(i,codes.numpy()[0,i,0],'o')

        i += 1

        plt.pause(0.001)
        
        #plt.savefig('out/img%d.png'%i)

In [106]:
animateCoordinates(IMGsIn, codesIn, modelIn)

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

...

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

In [107]:
animateCoordinates(IMGsOut, codesOut, modelOut)

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

...

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:12: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  if sys.path[0] == '':
C:\Users\allan\Anaconda3\envs\tf2-gpu\lib\site-packages\ipykernel_launcher.py:16: MatplotlibDeprecationWarning: Adding an axes using the same arguments as

In [ ]:
modelEncoderIn.trainable = False
modelDecoderOut.trainable = False

In [ ]:
timeSize = codesIn.shape[1]
U = codesIn.numpy() - np.mean(codesIn.numpy())
Y = decodesOut.numpy() - np.mean(codesOut.numpy())

In [ ]:
optimizer = tf.optimizers.Adam( learning_rate=0.005 )
loss = tf.keras.losses.MSE

modelDynamic.compile(loss=loss, optimizer=optimizer)

plotCallbacks = [PlotLearning()]

NStep = 1

modelDynamic.fit(U[:,::NStep,:], Y[:,::NStep,:], batch_size=1, epochs=200000, verbose=0, callbacks=plotCallbacks)

In [104]:
yHat = modelDynamic(U[:,::NStep,:]).numpy()
plt.plot(Y[0,::NStep,0])
plt.plot(yHat[0,:,0]) 

In [109]:
poles = np.linalg.eig(modelDynamic.layers[0].weights[1].numpy())[0]


A = modelDynamic.layers[0].weights[1].numpy()
B = modelDynamic.layers[0].weights[0].numpy()
C = modelDynamic.layers[1].weights[0].numpy()

print(A)
print(B)
print(C)

print('----')

print(poles)
print(np.abs(poles))
print(np.angle(poles))

[[ 0.9781466  -0.17435518]
 [ 0.42968708  0.94242865]]
[[0.0212182  0.24794236]]
[[0.31984848]
 [0.00464178]]
----
[0.96028763+0.2731286j 0.96028763-0.2731286j]
[0.99837446 0.99837446]
[ 0.2771061 -0.2771061]


In [111]:
Y

array([[[-0.04117066],
        [-0.02701241],
        [-0.07371002],
        [-0.08863264],
        [-0.13028675],
        [-0.10357672],
        [-0.07779795],
        [-0.0899592 ],
        [-0.09224087],
        [-0.10151726],
        [-0.13965613],
        [-0.18356615],
        [-0.1881476 ],
        [-0.17899615],
        [-0.12168747],
        [-0.08820826],
        [-0.0640164 ],
        [-0.03856808],
        [-0.0057798 ],
        [ 0.05576795],
        [ 0.10015911],
        [ 0.19670767],
        [ 0.28491396],
        [ 0.33972448],
        [ 0.39830345],
        [ 0.52771133],
        [ 0.6871037 ],
        [ 0.8839688 ],
        [ 1.0867519 ],
        [ 1.2396786 ],
        [ 1.388057  ],
        [ 1.473114  ],
        [ 1.5171671 ],
        [ 1.5103939 ],
        [ 1.4861853 ],
        [ 1.3928025 ],
        [ 1.1778126 ],
        [ 0.8755633 ],
        [ 0.5068056 ],
        [ 0.1738252 ],
        [-0.27238804],
        [-0.7091785 ],
        [-0.9880815 ],
        [-1

In [ ]:
model = tf.keras.models.Sequential( [modelEncoderIn, modelDynamic, modelDecoderOut] )
model.build( input_shape = modelEncoder.inputs[0].shape  )

optimizer = tf.optimizers.Adam(  learning_rate=0.02 )
loss = tf.keras.losses.MSE

model.compile(loss=loss, optimizer=optimizer)

In [ ]:
model.inputs

In [ ]:
plotCallbacks = [PlotLearning()]

model.fit(IMGsIn.reshape( (1, *IMGsIn.shape) ), IMGsOut.reshape( (1, *IMGsOut.shape) ), batch_size=1, epochs=196000, verbose=0, callbacks=plotCallbacks)

In [ ]:
IMGsOutPred = model(IMGsIn.reshape( (1, *IMGsIn.shape) ))

for imgsIn, imgsOut, imgsOutPred in zip(IMGsIn, IMGsOut, IMGsOutPred[0,:,:,:,:]):
    plt.subplot(1,3,1)
    plt.cla()
    plt.imshow(imgsIn)
    plt.subplot(1,3,2)
    plt.cla()
    plt.imshow(imgsOut)
    plt.subplot(1,3,3)
    plt.cla()
    plt.imshow(imgsOutPred)
    
    plt.pause(0.001)

In [ ]:
modelDynamic.layers[0].weights[1]

In [ ]:
u = [1.0,1.0,0,0,0,0,0,0,0,0,0,0]

_ = np.array(u).reshape((1,len(u),1))

codesPred = modelDynamic.predict(_)

plt.plot(codesPred[0])

In [ ]:
u = codesIn.numpy()
y = decodesOut.numpy()

Y = y[2:]
M = np.concatenate( ( -y[1:-1], -y[:-2], u[1:-1], u[:-2]   ), axis=1 )

x = np.linalg.pinv(M)@Y

In [ ]:
yHat = M@x

In [ ]:
plt.plot(y)
plt.plot(yHat)

In [ ]:
x

In [ ]:
print(modelDynamic.weights)

In [ ]:
modelDynamic.layers[1].set_weights( [np.array([[1 , 1.3]]) ,  np.array([[0.3, 0.2],[1.0, -0.4]] )] ) 
modelDynamic.layers[2].set_weights( [np.array([[1.0] , [0.5]])] ) 

In [ ]:
U.shape